In [1]:
%pip install accelerate peft bitsandbytes transformers trl

  Obtaining dependency information for peft from https://files.pythonhosted.org/packages/8b/1b/aee2a330d050c493642d59ba6af51f3910cb138ea48ede228c84c204a5af/peft-0.7.1-py3-none-any.whl.metadata
  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/9b/63/489ef9cd7a33c1f08f1b2be51d1b511883c5e34591aaa9873b30021cd679/bitsandbytes-0.42.0-py3-none-any.whl.metadata
  Obtaining dependency information for trl from https://files.pythonhosted.org/packages/61/ae/fb06164af1d535947067492f6db43446d984d1bfa7084f88dcae12ae7b48/trl-0.7.10-py3-none-any.whl.metadata
  Obtaining dependency information for tyro>=0.5.11 from https://files.pythonhosted.org/packages/b5/d2/0f8812ddc01f602f31489f1141f13100eef7b24f00a14b2fd27d9e8cbc97/tyro-0.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for shtab>=1.5.6 from https://files.pythonhosted.org/packages/40/ad/7227da64498eaa7abecee4311008f70869e156014b3270cec36e2e70cd31/shtab-1.6.5-py3-none-any.whl.metadata
  

In [2]:
!pip install datasets
!pip install gdown

  Obtaining dependency information for gdown from https://files.pythonhosted.org/packages/9e/db/c5dad38733f0995dea4480de38df1b3f6222b77b0dc89aad2402d86e2ff4/gdown-5.0.1-py3-none-any.whl.metadata


In [3]:
# Import required libraries
import textwrap
import pandas as pd
from datasets import load_dataset

import os
import json
import torch
import random
import zipfile
import transformers
import bitsandbytes as bnb

from datasets import Dataset
from huggingface_hub import HfApi
from datasets import load_dataset
from huggingface_hub import login
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM,prepare_model_for_int8_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, Trainer, TrainingArguments, DataCollatorForLanguageModeling, EarlyStoppingCallback, pipeline, logging, set_seed, TextStreamer , LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
!gdown --fuzzy -O NumEval_Task1.zip https://drive.google.com/file/d/1e09QNfGnzey42rf0Pwbk_ru3H5OLZo6N/view?usp=sharing

Downloading...
From (original): https://drive.google.com/uc?id=1e09QNfGnzey42rf0Pwbk_ru3H5OLZo6N
From (redirected): https://drive.google.com/uc?id=1e09QNfGnzey42rf0Pwbk_ru3H5OLZo6N&confirm=t&uuid=74244c1c-ca9c-462a-be0b-a7406f2a3ce9
To: /kaggle/working/NumEval_Task1.zip
100%|█████████████████████████████████████████| 113M/113M [00:00<00:00, 166MB/s]


In [5]:
zip_file_path = '/kaggle/working/NumEval_Task1.zip'
extracted_folder_path = '/kaggle/working/NumEval'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

In [6]:
base_model = "microsoft/Orca-2-7b"

In [7]:
list_of_file_train_QNLI= ['/kaggle/working/NumEval/NumEval_Task1/QNLI/QNLI-Stress Test/QNLI-Stress Test_train.json']
list_of_file_val_QNLI = ['/kaggle/working/NumEval/NumEval_Task1/QNLI/QNLI-Stress Test/QNLI-Stress Test_dev.json']
# list_of_file_test_QP_command = ['/kaggle/working/NumEval/NumEval_Task1/QP/Numeracy600K_comment_test.json',]


In [8]:
def give_prompt_QNLI(statement1,statement2,Question,
                     statement1_char,statement2_char,
                     statement1_sci_10E,statement2_sci_10E,
                     statement1_sci_10E_char,statement2_sci_10E_char,
                     statement1_mask,statement2_mask,answer):


    prompt = f"""
Find the number in the statements and then figure out the sentiment of the statement and then think about the Question and choose one of the options in the Question as answer and then say the  answer. know your explanation as value of explain then give me resualt in format: "{" Response: answer , Explanation : explain"}"
### Input:
statement1:{statement1}
statement2:{statement2}
### Question:
{Question}"""

    explain = f'''
first make numbers of statement1 to char it is equal to {statement1_char} then make numbers of statement2 to char it is equal to {statement2_char}.
seconde make statement1 to sci 10E char it is equal to {statement1_sci_10E_char} then make statement2 to sci 10E char it is equal to {statement2_sci_10E_char}.
third make statement1 sci 10E char to sci 10E it is equal to {statement1_sci_10E} then make statement2 sci 10E char to sci 10E it is equal to {statement2_sci_10E}.
Now we can mask the number and we now the value of ech of them and then we have statement1 is equal to {statement1_mask} and statement2 is equal to {statement2_mask}.
So after this steps we must figure out the sentiment of the statements and then think about the Question use the numbers to find out the answer.'''
      
    return {'prompt':prompt,'response':answer,'explanation':explain}

In [9]:
list_of_prompt_train_QNLI = []
list_of_prompt_val_QNLI = []
for file_name in list_of_file_train_QNLI:
    with open(file_name, 'r') as file:
        datas = json.load(file)
        for data in datas:
            list_of_prompt_train_QNLI.append(give_prompt_QNLI(data['statement1'],data['statement2'],data['options'],
                                                                    data['statement1_char'],data['statement2_char'],
                                                                    data['statement1_sci_10E'],data['statement2_sci_10E'],
                                                                    data['statement1_sci_10E_char'],data['statement2_sci_10E_char'],
                                                                    data['statement1_mask'],data['statement2_mask'],data['answer']))

for file_name in list_of_file_val_QNLI:
    with open(file_name, 'r') as file:
        datas = json.load(file)
        for data in datas:
            list_of_prompt_val_QNLI.append(give_prompt_QNLI(data['statement1'],data['statement2'],data['options'],
                                                                    data['statement1_char'],data['statement2_char'],
                                                                    data['statement1_sci_10E'],data['statement2_sci_10E'],
                                                                    data['statement1_sci_10E_char'],data['statement2_sci_10E_char'],
                                                                    data['statement1_mask'],data['statement2_mask'],data['answer']))

In [10]:
list_of_prompt_val_QNLI[2]

{'prompt': '\nFind the number in the statements and then figure out the sentiment of the statement and then think about the Question and choose one of the options in the Question as answer and then say the  answer. know your explanation as value of explain then give me resualt in format: " Response: answer , Explanation : explain"\n### Input:\nstatement1:Shaquan has more than 3 playing cards , each one is ordered by the number on it , but one card is flipped over . They are numbered 8 , 16 , 24 , x , 40\nstatement2:Shaquan has 5 playing cards , each one is ordered by the number on it , but one card is flipped over .\nThey are numbered 8 , 16 , 24 , x , 40\n### Question:\n Entailment or contradiction or neutral?',
 'response': 'neutral',
 'explanation': '\nfirst make numbers of statement1 to char it is equal to Shaquan has more than 3 playing cards , each one is ordered by the number on it , but one card is flipped over . They are numbered 8 , 1 6 , 2 4 , x , 4 0 then make numbers of st

In [11]:
len(list_of_prompt_train_QNLI)

6475

In [12]:
len(list_of_prompt_val_QNLI)

970

In [13]:
def QNLI_input_model(json_input):
    return f"""{json_input["prompt"]}
###explanation:
{json_input["explanation"]}
### Response:
{{response: {json_input["response"]}}}"""


In [14]:
list_train = []
list_val = []

# Shuffle
shuffled_prompt_train = random.sample(list_of_prompt_train_QNLI, len(list_of_prompt_train_QNLI))

for item in shuffled_prompt_train:
    list_train.append(QNLI_input_model(item))
 
# Shuffle
shuffled_prompt_val = random.sample(list_of_prompt_val_QNLI, len(list_of_prompt_val_QNLI))

for item in shuffled_prompt_val:
    list_val.append(QNLI_input_model(item))

In [15]:
len(list_train)

6475

In [16]:
len(list_val)

970

In [17]:
data_dict_train = {"train": list_train}
data_dict_val = {"val": list_val}

dataset_tarin = Dataset.from_dict(data_dict_train)
dataset_val = Dataset.from_dict(data_dict_val)

In [18]:
# orca2 attention dimension
lora_r = 16

# Alpha parameter for orca2 scaling
lora_alpha = 64

# Dropout probability for orca2 layers
lora_dropout = 0.1

# Bias
bias = "none"

# Task type
task_type = "CAUSAL_LM"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Batch size per GPU for training
per_device_train_batch_size = 20

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Optimizer to use
# optim = "paged_adamw_32bl"
optim = "paged_adamw_32bit"

# Number of training steps (overrides num_train_epochs)
max_steps = 100
num_train_epochs = 1

# Linear warmup steps from 0 to learning_rate
warmup_steps = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True

# Log every X updates steps
logging_steps = 1

In [19]:
def load_model(model_name, bnb_config):
    """
    Loads model and model tokenizer

    :param model_name: Hugging Face model name
    :param bnb_config: Bitsandbytes configuration
    """

    # Get number of GPU device and set maximum memory
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )

    # Load model tokenizer with the user authentication token
    tokenizer = AutoTokenizer.from_pretrained(model_name,add_eos_token=True)

    # Set padding token as EOS token
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [20]:
# Activate 4-bit precision base model loading
load_in_4bit = True

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute data type for 4-bit base models
bnb_4bit_compute_dtype = torch.bfloat16

In [21]:
def tokenize(prompt, add_eos_token=True):
        # there's probably a way to do this with the tokenizer settings
        # but again, gotta move fast
        result = tokenizer(
            prompt,
            truncation=True,
            max_length=cutoff_len,
            padding=False,
            return_tensors=None,
        )
        if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            and len(result["input_ids"]) < cutoff_len
            and add_eos_token
        ):
            result["input_ids"].append(tokenizer.eos_token_id)
            result["attention_mask"].append(1)

        result["labels"] = result["input_ids"].copy()

        return result

def generate_and_tokenize_prompt(data_point):
        keys = data_point.keys()
        
        for k in keys:
            key=k
            break
        tokenized_full_prompt = tokenize(data_point[k])

        return tokenized_full_prompt

In [22]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
    """
    Configures model quantization method using bitsandbytes to speed up training and inference

    :param load_in_4bit: Load model in 4-bit precision mode
    :param bnb_4bit_use_double_quant: Nested quantization for 4-bit model
    :param bnb_4bit_quant_type: Quantization data type for 4-bit model
    :param bnb_4bit_compute_dtype: Computation data type for 4-bit model
    """

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config

In [23]:
bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

model, tokenizer = load_model(base_model, bnb_config)

tokenizer.pad_token_id = 0 
tokenizer.padding_side = "left" 
cutoff_len = 512

config.json:   0%|          | 0.00/582 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/146 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/828 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [24]:
train_data = dataset_tarin.shuffle().map(generate_and_tokenize_prompt)
val_data = dataset_val.shuffle().map(generate_and_tokenize_prompt)

  0%|          | 0/6475 [00:00<?, ?ex/s]

  0%|          | 0/970 [00:00<?, ?ex/s]

In [25]:
def find_all_linear_names(model):
    """
    Find modules to apply orca2 to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"orca2 module names: {list(lora_module_names)}")
    return list(lora_module_names)

In [26]:
def print_trainable_parameters(model, use_4bit = False):
    """
    Prints the number of trainable parameters in the model.

    :param model: PEFT model
    """

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

In [27]:
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    """
    Creates Parameter-Efficient Fine-Tuning configuration for the model

    :param r: orca2 attention dimension
    :param lora_alpha: Alpha parameter for orca2 scaling
    :param modules: Names of the modules to apply orca2 to
    :param lora_dropout: Dropout Probability for orca2 layers
    :param bias: Specifies if the bias parameters should be trained
    """
    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

In [28]:
def fine_tune(model, tokenizer, dataset, eval_dataset, lora_r, lora_alpha, lora_dropout, bias, task_type, per_device_train_batch_size, gradient_accumulation_steps, warmup_steps, max_steps, num_train_epochs, learning_rate, fp16, logging_steps, output_dir, optim):

    """
    Prepares and fine-tune the pre-trained model.

    :param model: Pre-trained Hugging Face model
    :param tokenizer: Model tokenizer
    :param dataset: Preprocessed training dataset
    """

    # Enable gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # Prepare the model for training
    model = prepare_model_for_kbit_training(model)

    # Get orca2 module names
    target_modules = find_all_linear_names(model)

    # Create PEFT configuration for these modules and wrap the model to PEFT
    peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)

    # Training parameters
    trainer = Trainer(
        model = model,
        train_dataset = dataset,
        eval_dataset=val_data,
        args = TrainingArguments(
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
#             max_steps = max_steps,
            num_train_epochs = num_train_epochs,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            optim = optim,
        ),
        data_collator = transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True)
        )
    model.config.use_cache = False

    do_train = True

    # Launch training and log metrics
    print("Training...")

    trainer.train()
    model.save_pretrained(output_dir)
    torch.cuda.empty_cache()

In [29]:
fine_tune(model, tokenizer, train_data,val_data,lora_r, lora_alpha, lora_dropout, bias, task_type, per_device_train_batch_size, gradient_accumulation_steps, warmup_steps, max_steps, num_train_epochs, learning_rate, fp16, logging_steps, output_dir, optim)

LoRA module names: ['k_proj', 'gate_proj', 'o_proj', 'v_proj', 'q_proj', 'down_proj', 'up_proj']
All Parameters: 3,540,414,464 || Trainable Parameters: 39,976,960 || Trainable Parameters %: 1.1291604530062163
Training...


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.846700
2,1.884200
3,1.556800
4,1.211900
5,1.001600
6,0.753100
7,0.550900
8,0.407700
9,0.353600
10,0.290400


In [30]:
login()

In [31]:
api = HfApi()

api.upload_file(
    path_or_fileobj="/kaggle/working/results/adapter_config.json",
    path_in_repo="task7/task1/QNLI/fine_tune/COT_fine_tune/Orca_2_7b_task7-1_QNLI_COT_fine_tune_1epoch_adapter_config.json",
    repo_id="Mahmood1998/SEM_EVAL_2024",
    repo_type="model",
)

api.upload_file(
    path_or_fileobj="/kaggle/working/results/adapter_model.safetensors",
    path_in_repo="task7/task1/QNLI/fine_tune/COT_fine_tune/Orca_2_7b_task7-1_QNLI_COT_fine_tune_adapter_1epoch_model.safetensors",
    repo_id="Mahmood1998/SEM_EVAL_2024",
    repo_type="model",
)

adapter_model.safetensors:   0%|          | 0.00/160M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mahmood1998/SEM_EVAL_2024/commit/65f94fc62b26932ce260b92616575576ae2c97b3', commit_message='Upload task7/task1/QNLI/fine_tune/COT_fine_tune/Orca_2_7b_task7-1_QNLI_COT_fine_tune_adapter_1epoch_model.safetensors with huggingface_hub', commit_description='', oid='65f94fc62b26932ce260b92616575576ae2c97b3', pr_url=None, pr_revision=None, pr_num=None)